<a href="https://colab.research.google.com/github/RhysWangJunfei/nilm/blob/master/AMPDs/training/nilm_nn_woe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import tensorflow as tf
import numpy as np
import io
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split 
from google.colab import files
import matplotlib.pyplot as plt

In [0]:
#uploaded = files.upload()
'''Load data'''
#non_time_data = pd.read_csv(io.BytesIO(uploaded['Electricity_WHE.csv']))[['unix_ts']]
non_time_data = pd.read_csv('Electricity_WHE.csv')['unix_ts']

In [0]:
#uploaded = files.upload()
#CDE_data = pd.read_csv(io.BytesIO(uploaded['Electricity_CDE.csv']))['P']
WOE_data = pd.read_csv('Electricity_WOE.csv')['P']

In [0]:
#uploaded = files.upload()
#nn_data = pd.read_csv(io.BytesIO(uploaded['nn_data.csv']))
nn_data = pd.read_csv('nn_data.csv')
print(nn_data.shape)

In [0]:
window_size=60
non_time_data = pd.to_datetime(non_time_data,unit='s')
whe_month = non_time_data.dt.month.as_matrix()[window_size-1:].reshape([-1,1])
whe_hour = non_time_data.dt.hour.as_matrix()[window_size-1:].reshape([-1,1])
whe_weekday = non_time_data.dt.dayofweek.as_matrix()[window_size-1:].reshape([-1,1])
non_time = np.hstack([whe_month,whe_hour,whe_weekday])
non_time = np.concatenate([non_time[0:472500,:],non_time[475500:,:]],axis=0)

nn_data_ = np.concatenate([nn_data[0:472500],nn_data[475500:]],axis=0)

data_arr = np.hstack([nn_data_,non_time])

dataX = pd.get_dummies(pd.DataFrame(data_arr),columns=[3,4,5]).values
woe_Y = WOE_data.as_matrix()[window_size-1:].reshape([-1,1])
woe_Y = np.concatenate([woe_Y[0:472500,:],woe_Y[475500:,:]],axis=0)
categorized_woe_Y = np.ones(app_Y.shape)*1
categorized_woe_Y[[np.where(app_Y<800)[0]],:]=0
encoder = OneHotEncoder(handle_unknown='ignore',sparse=False)
cdeY_1hot = encoder.fit_transform(categorized_cde_Y)

In [0]:
X_train_cv, X_test, y_train_cv, y_test = train_test_split(\
            dataX,cdeY_1hot,\
            test_size=0.02, shuffle=True)
X_train, X_cv, y_train, y_cv = train_test_split(X_train_cv, y_train_cv, test_size=0.02, shuffle=True)

In [0]:
max_indice = np.argmax(y_train,1)
df = pd.Series(max_indice)
df.value_counts()

In [0]:
from imblearn.under_sampling import RandomUnderSampler
tl = RandomUnderSampler(sampling_strategy={0:9000,1:9000,2:9000})
X_train, y_train = tl.fit_resample(X_train, y_train)
y_train=encoder.transform(y_train)
whole_train = np.concatenate([X_train,y_train],axis=1)

In [0]:
'''Hyper parameters for deep learning'''
# Hyper Parameters
LR = 0.001               # learning rate
#cfg_list = nf.model_configs()
#error_list = []

#hyperparameters
batch_size=128
unit_num=128

In [0]:
#define the Feedforward Model
'''Feedforward Model Definition'''
tf.reset_default_graph()

''''''
#define inputs
tf_x = tf.placeholder(tf.float32, [None, 45],name='ff_x')
tf_y = tf.placeholder(tf.int32, [None, 2],name='ff_y')

l1 = tf.layers.dense(tf_x,64,activation=tf.nn.leaky_relu,name='ff_l1')
l2 = tf.layers.dense(l1,128,activation=tf.nn.leaky_relu,name='ff_l2')
l3 = tf.layers.dense(l2,256,activation=tf.nn.leaky_relu,name='ff_l3')
l4 = tf.layers.dense(l3,128,activation=tf.nn.leaky_relu,name='ff_l4')
l5 = tf.layers.dense(l4,64,activation=tf.nn.leaky_relu,name='ff_l5')
l6 = tf.layers.dense(l5,32,activation=tf.nn.leaky_relu,name='ff_l6')
l7 = tf.layers.dense(l6,8,activation=tf.nn.leaky_relu,name='ff_l7')
l8 = tf.layers.dense(l7,4,activation=tf.nn.leaky_relu,name='ff_l8')
pred = tf.layers.dense(l8,2,activation=tf.nn.relu,name='ff_pred')

with tf.name_scope('loss'):
    cross_entropy =  tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_y, logits=pred) 
    loss = tf.reduce_mean(cross_entropy)
    tf.summary.scalar("loss",tensor=loss)

train_op = tf.train.AdamOptimizer(LR).minimize(loss)

accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(tf_y, axis=1), tf.argmax(pred, axis=1)), tf.float32))

init_op = tf.group(tf.global_variables_initializer(), tf.local_variables_initializer()) 
saver = tf.train.Saver()


In [0]:
sess = tf.Session()
#sess.run(init_op)
saver.restore(sess, 'my_net/save_rnn.ckpt')

for j in range(0,3000):
    print('###iteration: '+str(j)+'###')
    batch_index = np.random.choice(27000,batch_size)
    batch_train = whole_train[batch_index,:]
    batch_X = batch_train[:,:-2].reshape([-1,45])
    batch_y = batch_train[:,-2:]
    sess.run(train_op,{tf_x:batch_X , tf_y:batch_y})
    cost_ = sess.run(loss,{tf_x:batch_X, tf_y:batch_y})
    print('train loss= %.4f' % cost_)
    if(j%99==0):
      acc_train = sess.run(accuracy,{tf_x:batch_X, tf_y:batch_y})
      print('train loss= %.4f' % cost_+', Acc=%.2f'% acc_train)
      acc_cv = sess.run(accuracy,{tf_x: X_cv.reshape([-1,45]), tf_y:y_cv})
      print('CV Acc=%.2f'% acc_cv)
      pre_cv = sess.run(pred,feed_dict={tf_x: X_cv.reshape([-1,45]), tf_y:y_cv})
      y_lables_argmax = tf.argmax(y_cv,axis=1)  
      y_pred_cv_argmax = tf.argmax(pre_cv,axis=1)
      confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_cv_argmax, num_classes=3)
      print(confusion.eval(session=sess,feed_dict={tf_x:X_cv.reshape([-1,45]), tf_y:y_cv}))
      save_path = saver.save(sess, "my_net/save_rnn.ckpt")
pre = sess.run(pred,feed_dict={tf_x: X_test.reshape([X_test.shape[0],45]), tf_y: y_test})
y_lables_argmax = np.argmax(y_test,1)
y_pred_argmax = np.argmax(pre,1)
confusion = tf.confusion_matrix(labels=y_lables_argmax, predictions=y_pred_argmax, num_classes=3)
#print('Confusion Matrix: \n\n', tf.Tensor.eval(confusion,feed_dict=None))
print(confusion.eval(session=sess))
sess.close()
